In [1]:
import torch
import torch.nn.functional as F
import json
from transformers import BertTokenizer, BertForMaskedLM

with open("./code/results.json", "r") as fp:
    results = json.load(fp)
    loss = results['loss']
    acc_single = results['acc_single']
    acc_pair = results['acc_pair']
    acc_topk = results['acc_topk']
    results = results['results']
    
print("      loss", loss)
print("acc_single", acc_single)
print("  acc_pair", acc_pair)
print("  acc_topk", acc_topk)

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForMaskedLM.from_pretrained('bert-base-chinese', return_dict = True)

      loss 0.10645139217376709
acc_single 0.9730933713471134
  acc_pair 0.9465431218816821
  acc_topk 0.9982181040627227


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
model = model.to('cuda:0')

In [10]:
@torch.no_grad()
def prob(w1='中', w2='国'):
    
    w1_token = tokenizer(w1, add_special_tokens=False, return_tensors = "pt")['input_ids']
    w2_token = tokenizer(w2, add_special_tokens=False, return_tensors = "pt")['input_ids']
    #print(w1_token, w2_token)
    
    mask = tokenizer.mask_token
    w1_given_w2 = tokenizer.encode_plus(mask+w2, return_tensors = "pt")
    w2_given_w1 = tokenizer.encode_plus(w1+mask, return_tensors = "pt")
    w1_given_w2 = {k: v.to('cuda:0') for k, v in w1_given_w2.items()}
    w2_given_w1 = {k: v.to('cuda:0') for k, v in w2_given_w1.items()}
    
    mask_index_w1_given_w2 = torch.where(w1_given_w2["input_ids"][0] == tokenizer.mask_token_id)
    mask_index_w2_given_w1 = torch.where(w2_given_w1["input_ids"][0] == tokenizer.mask_token_id)
    
    logits_w1_given_w2 = model(**w1_given_w2).logits[0, mask_index_w1_given_w2]
    logits_w2_given_w1 = model(**w2_given_w1).logits[0, mask_index_w2_given_w1]
    
    p_w1_given_w2 = F.softmax(logits_w1_given_w2, dim=-1)
    p_w2_given_w1 = F.softmax(logits_w2_given_w1, dim=-1)
    #print(p_w1_given_w2.shape)
    
    #val_w1_given_w2, pos_w1_given_w2 = torch.topk(p_w1_given_w2, k=5)
    #val_w2_given_w1, pos_w2_given_w1 = torch.topk(p_w2_given_w1, k=5)

    #print(val_w1_given_w2, val_w2_given_w1)
    #print(tokenizer.decode(pos_w1_given_w2[0]), tokenizer.decode(pos_w2_given_w1[0]))
    
    p_w1_given_w2 = p_w1_given_w2[0, w1_token]
    p_w2_given_w1 = p_w2_given_w1[0, w2_token]
    #print(p_w1_given_w2, p_w2_given_w1)
    
    return p_w1_given_w2, p_w2_given_w1



In [11]:
from tqdm import tqdm
import numpy as np

ans_t = []
for sample in tqdm(results):
    prob_dict = {}
    w1_candidates, w2_candidates = sample['char_pr'][0], sample['char_pr'][1]
    
    for w1, v1 in w1_candidates.items():
        for w2, v2 in w2_candidates.items():
            p_12, p_21 = prob(w1, w2)
            p = 0.5 * (p_12 * v2 + p_21 * v1)
            prob_dict[w1+w2] = p
    
    char_pr = max(prob_dict, key=prob_dict.get)
    char_gt = sample['char_gt']
    
    if char_pr == char_gt:
        ans_t.append(1)
    else:
        ans_t.append(0)

acc = np.array(ans_t).mean()
print(acc)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2806/2806 [29:07<00:00,  1.61it/s]

0.5007127583749109
